In [39]:
import numpy as np
import json
import os

In [42]:
def print_to_file(to_file, filename='model_selection'):
    f = open(filename, 'a')
    f.write(str(to_file))
    f.close()

In [49]:
data_dir = 'log/'
dirs = os.listdir(data_dir)
experiments = [x for x in dirs if '_' not in x]
for experiment in sorted(experiments):
    
    with open(data_dir + experiment + '/infos') as f:
        infos = json.loads(f.readline().replace("'", '"'))
        
    num_epochs = infos['num_epochs']
    with open(data_dir + experiment + '/performances') as f:
        performances = json.loads(f.readline().replace("'", '"'))

    print_to_file('Experiment: {}'.format(experiment))
    print_to_file('\nInfos: {}'.format(infos))
    
    try:
        with open(data_dir + experiment + '/exit_error') as f:
            exit_error = f.readline().replace('\n', '')
            print_to_file('\nExit error: {}'.format(exit_error))
    except:
        pass

    best_epoch = performances.index(max(performances))
    print_to_file('\nStatistical best epoch: {}'.format(best_epoch))
    
    with open(data_dir + experiment + '/confusion_matrices') as f:
        data = json.loads(f.readline().replace("'", '"'))

    conf_matrices = {}
    for epoch in range(num_epochs):
        conf_matrices[epoch] = data[str(epoch)]

    conf_matrix = conf_matrices[best_epoch]
    
    num_classes = 10
    num_samples = 0
    for i in range(num_classes):
        for j in range(num_classes):
            num_samples += conf_matrix[i][j]
            
    accuracy = 0
    for i in range(num_classes):
        accuracy += conf_matrix[i][i]
    accuracy /= num_samples

    print_to_file('\nAccuracy: {}'.format(accuracy)
    
    macro_precision = 0
    precision_total = [0] * num_classes
    precision = [0] * num_classes
    for i in range(num_classes):
        precision[i] = conf_matrix[i][i]
        for j in range(num_classes):
            precision_total[i] += conf_matrix[j][i]
        if precision_total[i] != 0:
            precision[i] /= precision_total[i]
        else:
            precision[i] = float('NaN')
        macro_precision += precision[i] / num_classes

    print_to_file('\nMacro precision: {}'.format(macro_precision)
    
    macro_recall = 0
    recall_total = [0] * num_classes
    recall = [0] * num_classes
    for i in range(num_classes):
        recall[i] = conf_matrix[i][i]
        for j in range(num_classes):
            recall_total[i] += conf_matrix[i][j]
        if recall_total[i] != 0:
            recall[i] /= recall_total[i]
        else:
            recall[i] = float('NaN')
        macro_recall += recall[i] / num_classes

    print_to_file('\nMacro recall (Trues Rate): {}'.format(macro_recall)
                  
    macro_fr = 0
    fr = [0] * num_classes
    fr_total = [0] * num_classes
    for i in range(num_classes):
        for j in range(num_classes):
            fr_total[i] += conf_matrix[i][j]
        fr[i] = fr_total[i] - conf_matrix[i][i]
        fr[i] /= fr_total[i]
    macro_fr += fr[i] / num_classes

    print_to_file('\nFalses Rate: {}'.format(macro_fr)

    macro_f = 0
    f = [0] * num_classes
    for i in range(num_classes):
        if precision[i] + recall [i] != 0:
            f[i] = 2 * precision[i] * recall[i] / (precision[i] + recall[i])
        else:
            f[i] = float('NaN')
        macro_f += f[i] / num_classes

    print_to_file('\nMacro F1: {}\n\n'.format(macro_f)
    
print('Statistics saved to file.')

Statistics saved to file.
